# Data Analysis Cycle

<center><img src='./img/DataAnalysis_CycleTS0.png' width='800'></center>
<center><img src='./img/DataAnalysis_Cycle1.png' width='800'></center>
<center><img src='./img/DataAnalysis_CycleTS1.png' width='800'></center>
<center><img src='./img/DataAnalysis_CycleTS2.png' width='800'></center>
<center><img src='./img/DataAnalysis_CycleTS5.png' width='800'></center>
<center><img src='./img/DataAnalysis_Cycle4.png' width='800'></center>
<center><img src='./img/DataAnalysis_Cycle5.png' width='800'></center>
<center><img src='./img/DataAnalysis_Cycle6.png' width='800'></center>
<center><img src='./img/DataAnalysis_Cycle7.png' width='800'></center>
<center><img src='./img/DataAnalysis_Cycle8.png' width='800'></center>
<center><img src='./img/DataAnalysis_Cycle9.png' width='800'></center>

# **Import Library:** 분석에 사용할 모듈 설치

In [2]:
# !pip install --upgrade pip
# !python -m pip install --user --upgrade pip

In [3]:
# Ignore the warnings
import warnings
# warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# System related and data input controls
import os

# Data manipulation and visualization
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows = 20
pd.options.display.max_columns = 20
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

# Modeling algorithms
# General
import statsmodels.api as sm
from scipy import stats

# Model selection
from sklearn.model_selection import train_test_split

# Evaluation metrics
from sklearn import metrics
# for regression
from sklearn.metrics import mean_squared_error,  mean_absolute_error, mean_absolute_percentage_error

# **Data Loading:** 분석에 사용할 데이터 불러오기

<center><img src='./img/Data_BikeSharingDemand.png' width='800'></center>

| **변수** 	| **설명** 	|
|-----|-----|
| **datetime** 	| 날짜시간정보 	|
| **season** 	| 계절 	|
| **holiday** 	| 휴일여부 	|
| **workingday** 	| 주중/주말 	|
| **weather** 	| 날씨 	|
| **temp** 	| 온도 	|
| **atemp** 	| 체감 온도 	|
| **humidity** 	| 습도 	|
| **windspeed** 	| 풍속도 	|
| **casual** 	| 비회원 대여수량 	|
| **registered** 	| 회원 대여수량 	|
| **count** 	| 총 대여수량 	|

In [4]:
# # Colab에서 실행
# # 아래 코드 실행 후 파일 선택을 눌러서 
# # 'Bike_Sharing_Demand_Full.csv' 파일을 선택해주면 로딩 완료
# from google.colab import files
# files.upload()

# # 아래 코드 실행 시 'Bike_Sharing_Demand_Full.csv' 데이터 출력 가능
# import pandas as pd
# df = pd.read_csv('Bike_Sharing_Demand_Full.csv')
# df

In [5]:
# PC에 저장된 데이터의 위치 확인
# 파이썬이 데이터 위치를 확인할 수 있게 코드로 변환
# pandas 사용하여 데이터 불러오기
# 불러온 데이터 이름은 통상 dataframe의 약자인 df로 사용하는 편
location = os.path.join('.', 'data', 'Bike_Sharing_Demand_Full.csv')
df = pd.read_csv(location)
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00,1,0,0,1,9.84,14.39,81,0.00,3,13,16
1,2011-01-01 1:00,1,0,0,1,9.02,13.63,80,0.00,8,32,40
2,2011-01-01 2:00,1,0,0,1,9.02,13.63,80,0.00,5,27,32
3,2011-01-01 3:00,1,0,0,1,9.84,14.39,75,0.00,3,10,13
4,2011-01-01 4:00,1,0,0,1,9.84,14.39,75,0.00,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
17374,2012-12-31 19:00,1,0,1,2,10.66,12.88,60,11.00,11,108,119
17375,2012-12-31 20:00,1,0,1,2,10.66,12.88,60,11.00,8,81,89
17376,2012-12-31 21:00,1,0,1,1,10.66,12.88,60,11.00,7,83,90
17377,2012-12-31 22:00,1,0,1,1,10.66,13.63,56,9.00,13,48,61


# **Preprocessing:** 데이터 정리 및 패턴 추출하기

- **목표:** 

    - 대량으로 수집된 데이터는 `그대로 활용 어려움`
    
    - `잘못 수집/처리 된 데이터`는 엉뚱한 결과를 발생
    
    - 알고리즘이 `학습이 가능한 형태`로 데이터를 정리

<img src='./img/DataAnalysis_Time.jpg' width='500'>


**일반적인 전처리 필요항목:**  

- 데이터 결합

- 결측값 처리

- 이상치 처리

- 자료형 변환

- `데이터 분리`

- `데이터 변환`

- 스케일 조정

$\Rightarrow$ **"알고리즘의 범위와 종류가 다양하여, `각 알고리즘의 입력에 맞게 변환`하는 것이 최선"**

## 시계열 변수 추출

#### 1) 시계열 데이터는 `여러 구성 성분`

- **시계열 = `체계적 성분` + 불규칙 성분**

<img src='./img/TimeSeries_Decomposition.png' width='600'>

| **시계열 방향** 	| **시계열 구성요소** 	| **의미** 	|
|-----|-----|-----|
| **장기적 움직임** 	| **추세(Trend)** 	| 장기적으로 시간흐름에 따른 데이터의 상승/하강 경향 	|
| **단기적 움직임** 	| **계절성(Seasonality)** 	| 특정기간 간격으로 반복적으로 나타나는 패턴 	|
| | **주기(Cycle)** 	| 계절변동으로 설명되지 않는 패턴(기후변화, 정책변화, 사회관습 등) 	|
| **불규칙 움직임** 	| **에러(Error)** 	| 사전(Prior)적으로 예측할 수 없는 특수 패턴(지진, 전쟁, 파업 등) 	|

#### 2) 여러 구성성분들이 `단기적 및 장기적`으로 복합적으로 융합되어 작동

<img src='img/TS_Example3.png' width='600'>(http://www.sbr.ai/news/articleView.html?idxno=1608)

In [6]:
# 데이터의 각 column의 타입을 확인하기
# df.shape
# df.ndim
# df.head()
# df.tail()
# df.describe(include='all').T
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    17379 non-null  object 
 1   season      17379 non-null  int64  
 2   holiday     17379 non-null  int64  
 3   workingday  17379 non-null  int64  
 4   weather     17379 non-null  int64  
 5   temp        17379 non-null  float64
 6   atemp       17379 non-null  float64
 7   humidity    17379 non-null  int64  
 8   windspeed   17379 non-null  float64
 9   casual      17379 non-null  int64  
 10  registered  17379 non-null  int64  
 11  count       17379 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1.6+ MB


In [7]:
# 데이터 수치 확인
df

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 0:00,1,0,0,1,9.84,14.39,81,0.00,3,13,16
1,2011-01-01 1:00,1,0,0,1,9.02,13.63,80,0.00,8,32,40
2,2011-01-01 2:00,1,0,0,1,9.02,13.63,80,0.00,5,27,32
3,2011-01-01 3:00,1,0,0,1,9.84,14.39,75,0.00,3,10,13
4,2011-01-01 4:00,1,0,0,1,9.84,14.39,75,0.00,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
17374,2012-12-31 19:00,1,0,1,2,10.66,12.88,60,11.00,11,108,119
17375,2012-12-31 20:00,1,0,1,2,10.66,12.88,60,11.00,8,81,89
17376,2012-12-31 21:00,1,0,1,1,10.66,12.88,60,11.00,7,83,90
17377,2012-12-31 22:00,1,0,1,1,10.66,13.63,56,9.00,13,48,61
